Librerías

In [514]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

from sklearn.metrics import mean_absolute_error, mean_squared_error

from prophet import Prophet
from prophet.diagnostics import generate_cutoffs, cross_validation, performance_metrics
from prophet.plot import plot_plotly
import plotly.graph_objects as go

import time


- Ce carga un subdataset con los datos necesarios para el modelo de ML

In [515]:
# Cargar los datos
df = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Input.csv")
df['date'] = pd.to_datetime(df['date'])

In [516]:
# Obtener el valor máximo de la columna 'date'
max_date = df['date'].max()
# Calcular la fecha de diciembre dentro de 5 años
future_date = datetime(max_date.year + 5, 12, 1)
# Calcular la cantidad de meses entre ambas fechas
months_difference = (future_date.year - max_date.year) * 12 + (future_date.month - max_date.month)

In [517]:
# Función para guardar el forecast en un archivo CSV
def guardar_predicciones(forecast, nombre_archivo="predicciones.csv"):
    try:
        forecast.to_csv(nombre_archivo, index=False)
        print(f"Predicciones guardadas en {nombre_archivo}")
    except Exception as e:
        print(f"Error al guardar las predicciones: {e}")

In [518]:
# # Definir parámetros de selección para la industria y la columna
# industry_type = input(f"Seleccione el tipo de industria para predecir {tuple(df['industry'].unique())}: ")
# column_name = input(f"Seleccione la variable que desea predecir ('total_trips','unique_vehicles','avg_hours_per_day_per_driver', 'total_amount', 'total_co2_emission'): ")

# # Selección de períodos y frecuencia para predicción
# periodos = int(input(f"La cantidad de meses entre {max_date.strftime('%Y-%m-%d')} y diciembre dentro de 5 años es: {months_difference} meses."
#                         "Seleccione la cantidad de períodos para la predicción: "))
# frecuencia = "M"

In [519]:
def graficar_original(df_prophet, column_name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], marker=dict(symbol='circle', color='royalblue')))
    fig.layout.update(title_text="Datos históricos", yaxis_title=f"{column_name}", xaxis_rangeslider_visible=True)
    fig.show()

In [520]:
def graficar_predicción(df_prophet, column_name, forecast,industry_type):
            
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], name='Datos Históricos', marker=dict(symbol='circle', color='royalblue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicción', marker=dict(symbol='diamond', color='yellow')))
    fig.add_trace(go.Scatter(x=forecast['ds'].tolist() + forecast['ds'][::-1].tolist(), 
                                y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'][::-1].tolist(), 
                                fill='toself', fillcolor='rgba(255, 255, 255, 0.2)', 
                                line=dict(color='rgba(255, 255, 255, 0)'), name='Intervalo de predicción'))
    fig.layout.update(xaxis_title='Fecha', yaxis_title=column_name, title_text=f"Predicción para {column_name} ({industry_type})", 
                        xaxis_rangeslider_visible=True)
    fig.show()

# Análisis para cada variable y cada industria

In [521]:
# Selección de períodos y frecuencia para predicción
industry_types = df['industry'].unique()
columns_to_predict = ['total_trips', 'unique_vehicles', 'avg_hours_per_day_per_driver', 'total_amount', 'total_co2_emission']
periodos = 64
frecuencia = "M"

In [522]:
df.head()

,date,industry,total_trips,unique_vehicles,total_amount,avg_trip_distance,avg_hours_per_day_per_driver,total_co2_emission,days_in_month
0,2021-01-01,FHV - High Volume,11902481,47594,1.917688e+08,3.64,6.800,17332.2978,31
1,2021-01-01,FHV - Other,1142350,10128,NaN,3.63,3.800,1660.0704,31
2,2021-01-01,Green Taxi,76477,982,1.748679e+06,3.46,4.000,105.8211,31
3,2021-01-01,Total Mercado,14486920,63329,2.148706e+08,3.29,5.475,20424.6565,31
4,2021-01-01,Yellow Taxi,1365612,4625,2.135305e+07,2.43,7.300,1326.4672,31


In [523]:
# Función para cargar los datos y filtrar la serie de tiempo seleccionada
def cargar_y_preparar_datos(df, industry_type, column_name):
    """
    Filtra y prepara los datos para Prophet según el tipo de industria y la columna seleccionada.
    """
    df_filtered = df[df['industry'] == industry_type][['date', column_name]].copy()
    df_filtered.columns = ['ds', 'y']  # Renombrar columnas para Prophet
    df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])  # Asegurar formato de fecha
    return df_filtered if not df_filtered['y'].isnull().all() else None

In [524]:
def evaluate_model(model, periodos, frecuencia):
    # Configuración de validación cruzada

    # This cross validation procedure can be done automatically for a range of historical cutoffs using the cross_validation function.
    # We specify the forecast horizon (horizon), and then optionally the size of the initial training period (initial) and the spacing
    # between cutoff dates (period). By default, the initial training period is set to three times the horizon, and cutoffs are made every half a horizon.
    # Dado que el set de datos tiene únicamente 44 meses de historia, no se puede aplicar cross validation con la cantidad de períodos necesarios (64),
    # por este motivo, se realiza el ajuste con 12 meses de pronóstico y 24 para el período de entrenamiento.

    initial = f"{24 * 30.4} days"  # 24 meses aproximados en días
    period = f"{30.4} days"        # Un mes aproximado en días
    horizon = f"{12 * 30.4} days"  # Horizonte basado en la entrada

    # Validación cruzada
    df_cv = cross_validation(
        model,
        initial=initial,
        period=period,
        horizon=horizon
    )
    
    # Extraer valores reales y predichos
    y_true = df_cv['y'].values
    y_pred = df_cv['yhat'].values

    # Calcular métricas
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    smape = 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    print("Resultados de validación cruzada:")    
    print(f'Mean Absolute Error: {mae:.2f}')
    print(f'Mean Absolute Percentage Error : {mape:.2f}')
    print(f'Symmetric Mean Absolute Percentage Errorr : {smape:.2f}')
    print(f'Mean Squared Error: {mse:.2f}')
    print(f'Root Mean Squared Error: {rmse:.2f}')

    return {'mae': mae, 'mape': mape, 'smape': smape, 'mse': mse, 'rmse': rmse}


In [525]:
def pronóstico_con_grid_search(df_prophet, periodos, frecuencia):
    """
    Realiza Grid Search para encontrar los mejores hiperparámetros de Prophet y calcula el error.
    """

    grid_search = {
        'changepoint_prior_scale': [0.01, 0.1, 0.5],
        'seasonality_prior_scale': [5.0, 10.0, 20.0],
        'seasonality_mode': ['additive', 'multiplicative'],
        'fourier_order': [5, 10, 20]
    }
    
    best_params = None
    best_error = float('inf')
    resultados = []
    
    # Grid Search
    for cps in grid_search['changepoint_prior_scale']:
        for sps in grid_search['seasonality_prior_scale']:
            for sm in grid_search['seasonality_mode']:
                for fo in grid_search['fourier_order']:
                      
                        print(f"Evaluanco combinación: CPS={cps}, SPS={sps}, SM={sm}, FO={fo}")
                        
                        # Crear y entrenar modelo Prophet con los parámetros actuales
                        model = Prophet(changepoint_prior_scale=cps, seasonality_prior_scale=sps, seasonality_mode=sm)
                        model = Prophet(weekly_seasonality=False)
                        model.add_seasonality(name='monthly', period=12, fourier_order=fo)
                        model.fit(df_prophet)

                        # Evaluar modelo
                        metrics = evaluate_model(model, periodos, frecuencia)
                        if metrics is None:
                            continue

                        resultados.append({
                            'changepoint_prior_scale': cps,
                            'seasonality_prior_scale': sps,
                            'seasonality_mode': sm,
                            'fourier_order': fo,
                            **metrics
                        })

                        # Actualizar mejores parámetros si el MAE es menor
                        if metrics['mae'] < best_error:
                            best_error = metrics['mae']
                            best_params = {
                                'changepoint_prior_scale': cps,
                                'seasonality_prior_scale': sps,
                                'seasonality_mode': sm,
                                'fourier_order': fo
                            }
                    

    return best_params, best_error, resultados

In [526]:
# Iterar sobre todas las combinaciones de industria y columna
BANDERA = True
mejores_resultados = {}
for industry_type in industry_types:
    for column_name in columns_to_predict:
        # Preparar los datos
        df_prophet = cargar_y_preparar_datos(df, industry_type, column_name)
        if BANDERA == True:
            if df_prophet is not None and not df_prophet.empty and df_prophet['y'].notnull().all():
                print(f"Procesando: {industry_type}, columna: {column_name}")
                start_time = time.time()

                # Ejecutar Grid Search
                best_params, best_error, resultados = pronóstico_con_grid_search(df_prophet, periodos=12, frecuencia='M')
                
                # Guardar resultados
                mejores_resultados[f"{industry_type}_{column_name}"] = {
                    'mejores_parametros': best_params,
                    'mejor_error': best_error,
                    'resultados_grid_search': resultados
                }
                    
                elapsed_time = time.time() - start_time
                print(f"Tiempo para {industry_type}, {column_name}: {elapsed_time:.2f} segundos")
                BANDERA = False


# Mostrar los mejores parámetros por combinación
for key, value in mejores_resultados.items():
    print(f"Industria y columna: {key}")
    print(f"Mejores parámetros: {value['mejores_parametros']}")
    print(f"Mejor error (MAE): {value['mejor_error']}\n")

    # Mostrar los primeros resultados del Grid Search
    print(value['resultados_grid_search'].head())


resumen_resultados = pd.DataFrame([
    {'industria_columna': key, **value['mejores_parametros'], 'mejor_error': value['mejor_error']}
    for key, value in mejores_resultados.items()
])
resumen_resultados.to_csv("resumen_mejores_resultados.csv", index=False)

Procesando: FHV - High Volume, columna: total_trips
Evaluanco combinación: CPS=0.01, SPS=5.0, SM=additive, FO=5


20:11:23 - cmdstanpy - INFO - Chain [1] start processing
20:11:24 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/8 [00:00<?, ?it/s]

20:11:25 - cmdstanpy - INFO - Chain [1] start processing


KeyboardInterrupt: 